In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.offline as pyo
from dateutil import parser
from Strategies import AmipyNF
from kiteconnect import KiteConnect
from Brokers import apikey

import pandas as pd
from kiteconnect import KiteConnect
import undetected_chromedriver as uc
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from datetime import datetime
import time, pyotp

import logging
from kiteconnect import KiteTicker
from kiteconnect import KiteConnect
from Brokers import apikey
from datetime import datetime
import time

In [2]:
tokens = [256265]
acctkn_file = r'Brokers\acc_token.txt'
reqtkn_file = r'Brokers\req_token.txt'
kite_access_token = open(acctkn_file,'r').read()
kite_request_token = open(reqtkn_file,'r').read()
kite = KiteConnect(apikey.kite_api_key)
kite.set_access_token(kite_access_token)

In [6]:
from_date = '2023-05-01'
to_date = '2023-05-23'
interval = 'minute'
nf_hist_data = kite.historical_data(tokens[0], from_date, to_date, interval)
# Convert historical_data to pandas dataframe
nf_df = pd.DataFrame(nf_hist_data)
nf_df = nf_df.set_index('date')
nf_df.to_csv('nf_kite_3.csv')

In [8]:
nf_kite = pd.read_csv('nf_kite.csv')
nf_kite_2 = pd.read_csv('nf_kite_2.csv')
nf_kite_3 = pd.read_csv('nf_kite_3.csv')

nf_kite = pd.concat([nf_kite, nf_kite_2, nf_kite_3])

nf_kite = nf_kite.set_index('date')
nf_kite.to_csv('nf_kite_consolidated.csv')


In [ ]:
import glob
import pandas as pd

def concat_nifty_files(folder_path, output_csv):
    # Column names
    column_names = ['Ticker', 'Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Vol', 'OI']
    
    # Initialize an empty list to hold dataframes
    df_list = []

    # Loop through the years you are interested in
    for year in range(2019, 2024):  # replace 2024 with current year + 1
        # Pattern to match NIFTY files
        file_pattern = f'{folder_path}/{year}*NIFTY.txt'

        # Get all the matching file names
        nifty_files = glob.glob(file_pattern)
    
        # Loop through all the files and read them into dataframes
        for file in nifty_files:
            print(f"Processing file: {file}")
            try:
                df = pd.read_csv(file, header=None)
                df.columns = column_names
                df_list.append(df)
            except Exception as e:
                print(f"Error encountered while processing file: {file}")
                print(f"Error message: {str(e)}")
                continue

    # Concatenate all the dataframes
    combined_df = pd.concat(df_list, ignore_index=True)
    
    # Convert the 'Date' column to datetime
    combined_df['Date'] = pd.to_datetime(combined_df['Date'], format='%Y%m%d')
    
    # Sort by date and time
    combined_df = combined_df.sort_values(by=['Date', 'Time'])

    # Write the dataframe to a csv file
    combined_df.to_csv(output_csv, index=False)

# Call the function
folder_path = r"C:\Users\user\Downloads\Intraday 1 Min Data\Consolidated"
output_csv = 'nifty_data.csv'  # output csv file name
concat_nifty_files(folder_path, output_csv)


In [12]:
import pandas as pd
from datetime import datetime

# Load the data from the CSV files
df1 = pd.read_csv('nifty_data.csv')
df2 = pd.read_csv('nf_kite_consolidated.csv')

# Remove 'OI' column from df1
df1.drop(columns=['OI'], inplace=True)

# Convert 'Date' and 'Time' in df1 to datetime format and combine them
df1['DateTime'] = pd.to_datetime(df1['Date'] + ' ' + df1['Time'])

# Convert 'date' in df2 to datetime format
df2['date'] = pd.to_datetime(df2['date']).dt.tz_localize(None)  # removing timezone info

# Rename columns in df2 to match those in df1
df2.rename(columns={'date':'DateTime', 'open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'volume':'Vol'}, inplace=True)

# Merge the two dataframes on 'DateTime'
df = pd.merge(df1, df2, on='DateTime', how='outer')

# Write the merged dataframe to a new CSV file
df.to_csv('nf_backtest_consolidated.csv', index=False)


In [14]:
import pandas as pd

# Load the data from the CSV file
df = pd.read_csv('nf_backtest_consolidated.csv')

# Convert 'DateTime' to datetime format
df['DateTime'] = pd.to_datetime(df['DateTime'])

# Extract date and time from 'DateTime'
df['Date'] = df['DateTime'].dt.date
df['Time'] = df['DateTime'].dt.time

# Filter for rows where the time is 9:19
df_919 = df[df['Time'] == pd.to_datetime('09:19').time()]

# Create a new DataFrame containing just the date and 'Open' price at 9:19 for each date
df_start_price = df_919[['Date', 'Open']].copy()
df_start_price.columns = ['date', 'start_price']

# Write the new DataFrame to a CSV file
df_start_price.to_csv('nifty_919.csv', index=False)
